In [1]:
### INITIALISATION ###

import numpy as np          # fundamental package for scientific computing
import xarray as xr         # data handling
import glob                 # return all file paths that match a specific pattern
import os                   # provide operating system-related functions
import pop_tools            # to mask region of interest
import gsw                  # compute potential density

path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/vvel/'
salt_path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/salt/'
temp_path = '/Data/gfi/share/ModData/CESM2_LENS2/ocean/monthly/temp/'
files = glob.glob(path + '*.nc')

grid_name = 'POP_gx1v7'

#setting up of regional mask
region_defs = {
    'NorthAtlantic':[{'match': {'REGION_MASK': [6]}, 'bounds': {'TLAT': [-20.0, 66.0]}}],
    'LabradorSea': [{'match': {'REGION_MASK': [8]}, 'bounds': {'TLAT': [45.0, 66.0]}}],
    'MediterraneanSea': [{'match': {'REGION_MASK': [7]}}]} 
mask3d = pop_tools.region_mask_3d(grid_name, region_defs=region_defs, mask_name='North Atlantic')
mask3d = mask3d.sum('region')

i=0

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
import time

In [13]:
start_time = time.time()
ds = xr.open_dataset(files[i]).isel(time=slice(0,100)).where(mask3d == 1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 22.25760054588318 seconds ---


In [14]:
ds.VVEL.values.shape

(100, 60, 384, 320)

In [17]:
import numpy.ma as ma
mask = mask3d.values
mask4d = np.tile(mask, [100, 60, 1, 1])

In [18]:
start_time2 = time.time()
ds_numpy = xr.open_dataset(files[i]).isel(time=slice(0,100)).VVEL.values
mx = ma.masked_array(ds_numpy, mask=mask4d)
print("--- %s seconds ---" % (time.time() - start_time2))

--- 17.71786117553711 seconds ---
